# Encoder

<img src="https://machinelearningmastery.com/wp-content/uploads/2021/08/attention_research_1.png"  width="400" height="600">

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math

In [2]:
input_ids = torch.randint(high=1000, size=(1, 10,)) # The extra dimension at the start is for batch size.
input_ids

tensor([[238, 949, 643, 452,  18, 212, 795, 311, 345, 288]])

☝🏼 Assume that these are the token ids for some input sentence

In [3]:
vocab_size = 1000 # This is how many tokens do we have in our vocabulary
embed_dim = 4 # This is how many dimensions do we have
seq_len = 10
batch_size = 1
token_embedding = nn.Embedding(vocab_size, embed_dim)
position_embedding = nn.Embedding(seq_len, embed_dim)

In [4]:
positions = torch.arange(0, seq_len, dtype=torch.long)
pos_embed = position_embedding(positions)
print(pos_embed)
print('='*20)
print(pos_embed.size())

tensor([[-1.9662, -0.4666, -0.0085,  0.5426],
        [ 1.2209,  1.1104,  1.2040, -0.6781],
        [ 0.6317,  0.6101, -1.6729,  1.3828],
        [ 0.4638,  2.0086, -0.5469,  0.2994],
        [ 0.3300,  0.0974, -0.2671,  1.1689],
        [ 0.9100, -0.0302, -0.6700,  0.6310],
        [ 1.3327,  1.2089,  0.0128, -0.2166],
        [-0.8791,  0.3556,  1.4652, -0.2835],
        [-0.2674,  0.7822, -0.0789,  1.1939],
        [-0.4484, -0.0478,  1.5053,  1.8661]], grad_fn=<EmbeddingBackward0>)
torch.Size([10, 4])


In [5]:
input_embed = token_embedding(input_ids)
print(input_embed)
print('='*20)
print(input_embed.size())

tensor([[[ 4.1199e-01,  6.8473e-02, -7.4899e-01, -1.2729e+00],
         [ 3.7485e-01,  4.2052e-01,  5.6338e-01, -1.4777e+00],
         [-7.1622e-01, -4.5519e-01,  7.5808e-01,  1.6018e+00],
         [-1.0683e-03, -1.2865e+00,  6.1994e-01, -1.3466e+00],
         [ 5.9406e-02,  2.7505e+00, -4.5265e-02, -1.1146e+00],
         [ 1.5855e-01, -1.3531e+00, -2.1985e-01, -2.2839e-01],
         [-1.8950e+00,  3.3590e-01, -3.9597e-01, -7.2173e-01],
         [ 1.6348e+00,  9.0691e-01,  1.9599e+00,  1.8205e-01],
         [ 1.6333e-01, -9.8955e-01, -1.2956e+00, -1.0693e+00],
         [ 1.3906e+00, -3.5561e-01,  1.2018e+00,  1.3693e+00]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([1, 10, 4])


In [6]:
input_embed = input_embed + pos_embed
print(input_embed)
print('='*20)
print(input_embed.size())

tensor([[[-1.5542, -0.3981, -0.7575, -0.7303],
         [ 1.5957,  1.5309,  1.7674, -2.1558],
         [-0.0845,  0.1549, -0.9148,  2.9846],
         [ 0.4628,  0.7222,  0.0730, -1.0472],
         [ 0.3894,  2.8478, -0.3124,  0.0543],
         [ 1.0685, -1.3834, -0.8899,  0.4026],
         [-0.5623,  1.5448, -0.3832, -0.9383],
         [ 0.7558,  1.2625,  3.4250, -0.1014],
         [-0.1041, -0.2073, -1.3745,  0.1245],
         [ 0.9422, -0.4034,  2.7071,  3.2354]]], grad_fn=<AddBackward0>)
torch.Size([1, 10, 4])


☝🏼 Here we can see that each input token was converted into a vector of 4 floats. This is because we mentioned our `embed_dim = 4` before.

Hence the size of the input embeddings is: 10 x 4 i.e. 10 tokens by 4 embedding dimensions

---

Usually people would add a positional encoding at this step, but we are going for Alibi mechanism so we can skip the positional encodings for now.

---

In [5]:
residual_embeddings_1 = input_embed.clone() # This will be used in the addition step after we perform attention

## Attention ‼

> We perform normalization before attention mechanism as it has shown to produce better results.
>
> ~ Source: nanoGPT

In [ ]:
# For normalization we actually have to use trainable parameters
epsilon = nn.Parameter(torch.ones(embed_dim))
gamma = nn.Parameter(torch.zeros(embed_dim))

input_embed = F.layer_norm(input_embed, epsilon.shape, epsilon, gamma, 1e-5)

In [6]:
# This Linear layer will help us split our embeddings into Query, Key, and Value
qkv_layer = nn.Linear(embed_dim, 3 * embed_dim, bias=False)

q, k, v = qkv_layer(input_embed).split(embed_dim, dim=2)
print(q.size(), k.size(), v.size())

torch.Size([1, 10, 4]) torch.Size([1, 10, 4]) torch.Size([1, 10, 4])


Now we have split our embeddings into Query Key and Value. 

## Multi Head Self Attention

In [8]:
# Let's say we have 3 heads in out multi head attention
n_head = 2 # IMP: Embedding dimension should be divisible by number of heads

print(q)

k = k.view(batch_size, seq_len, n_head, embed_dim // n_head).transpose(1, 2) # (Batch size, num_heads, sequence length, embeddings per head)
q = q.view(batch_size, seq_len, n_head, embed_dim // n_head).transpose(1, 2) # (Batch size, num_heads, sequence length, embeddings per head)
v = v.view(batch_size, seq_len, n_head, embed_dim // n_head).transpose(1, 2) # (Batch size, num_heads, sequence length, embeddings per head)

print(q)

tensor([[[ 6.1650e-01,  4.8034e-01, -2.9523e-01, -1.3133e-01],
         [-2.4997e-01, -1.4270e+00, -1.0238e+00,  6.7965e-01],
         [ 1.0477e-03,  5.8367e-01,  7.9020e-01, -5.9298e-02],
         [ 7.3255e-02, -2.2359e-01, -4.0985e-01, -3.1605e-01],
         [-5.9298e-01, -8.6120e-01, -3.8127e-01,  1.6565e-01],
         [-1.9287e-01, -5.7012e-01, -2.2677e-01, -1.4374e-01],
         [ 4.7553e-01,  6.4546e-01,  2.2596e-01,  4.1298e-01],
         [-6.4118e-01, -6.4194e-01,  1.6476e-01, -1.2222e-01],
         [ 7.9794e-01,  1.0585e+00, -2.7348e-02, -3.8212e-01],
         [ 1.9492e-02, -2.9926e-01, -2.0691e-01, -3.6282e-02]]],
       grad_fn=<SplitBackward0>)
tensor([[[[ 6.1650e-01,  4.8034e-01],
          [-2.4997e-01, -1.4270e+00],
          [ 1.0477e-03,  5.8367e-01],
          [ 7.3255e-02, -2.2359e-01],
          [-5.9298e-01, -8.6120e-01],
          [-1.9287e-01, -5.7012e-01],
          [ 4.7553e-01,  6.4546e-01],
          [-6.4118e-01, -6.4194e-01],
          [ 7.9794e-01,  1.0585

**☝🏼 As we can see, the Query embeddings got split into 3 parts across the embedding dimension**

## Attention Formula

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*dSwckeG028obZPWafgJrmw.png" width="350" height="150">

In [10]:
attention = (q @ k.transpose(-2, -1)) / math.sqrt(embed_dim // n_head) # In multi-head attention, Dk becomes the dimension of embeddings per head

print(attention.size())

# Here we should insert our Alibi mask

attention = F.softmax(attention, dim=-1)

print(attention.size())

new_embeddings = attention @ v
new_embeddings = new_embeddings.transpose(1, 2).contiguous().view(batch_size, seq_len, embed_dim) # re-assemble all head outputs side by side

print(new_embeddings)
print(new_embeddings.size())

torch.Size([1, 2, 10, 10])
torch.Size([1, 2, 10, 10])
tensor([[[ 0.1944,  0.2144,  0.0139,  0.0598],
         [ 0.0474,  0.0354,  0.0918,  0.0357],
         [ 0.1576,  0.1683, -0.2388,  0.0983],
         [ 0.1237,  0.1268,  0.0618,  0.0611],
         [ 0.0558,  0.0514,  0.0024,  0.0517],
         [ 0.0911,  0.0890, -0.0011,  0.0618],
         [ 0.1925,  0.2114, -0.1580,  0.0624],
         [ 0.0633,  0.0612, -0.0960,  0.0735],
         [ 0.2354,  0.2678, -0.0240,  0.0732],
         [ 0.1167,  0.1185, -0.0172,  0.0600]]], grad_fn=<ViewBackward0>)
torch.Size([1, 10, 4])


In [13]:
# Now let's add

new_embeddings = new_embeddings + residual_embeddings_1

print(new_embeddings)

tensor([[[-0.6722,  0.6018, -1.2264,  1.2969],
         [ 1.0653, -1.6424,  0.3096,  0.2675],
         [-0.2335,  1.6278, -1.0995, -0.2947],
         [-0.7587, -1.1540,  0.5934,  1.3193],
         [ 0.4369, -1.0010,  1.4327, -0.8686],
         [ 0.2858, -1.6500,  0.3179,  1.0463],
         [ 0.6036,  1.1452, -1.5048, -0.2440],
         [ 0.2643, -1.3342,  1.4231, -0.3532],
         [-1.0582,  1.3731, -0.8449,  0.5300],
         [ 0.4518, -1.4801, -0.2195,  1.2478]]],
       grad_fn=<NativeLayerNormBackward0>)


In [14]:
residual_embeddings_2 = new_embeddings.clone() # Again we keep track inorder to add them later

Again we do pre-normalization

In [ ]:
# For normalization we actually have to use trainable parameters
epsilon = nn.Parameter(torch.ones(embed_dim))
gamma = nn.Parameter(torch.zeros(embed_dim))

new_embeddings = F.layer_norm(new_embeddings, epsilon.shape, epsilon, gamma, 1e-5)

In [16]:
# The Feed-forward part is responsible to help the model learn more

make_big = nn.Linear(embed_dim, 4 * embed_dim, bias=False)
gelu = nn.GELU()
make_small = nn.Linear(4 * embed_dim, embed_dim, bias=False)

new_embeddings = make_big(new_embeddings)
new_embeddings = gelu(new_embeddings)
new_embeddings = make_small(new_embeddings)

The whole point of making it **BIG** and then back _small_ is because we hope that this step will allow the model to learn more information

In [17]:
new_embeddings = new_embeddings + residual_embeddings_2

print(new_embeddings)

tensor([[[-0.6929,  0.6125, -1.2365,  1.2892],
         [ 1.0700, -1.6340,  0.3206,  0.3252],
         [-0.3223,  1.7031, -1.1286, -0.3031],
         [-0.7291, -1.1515,  0.6072,  1.3607],
         [ 0.4585, -1.0053,  1.4406, -0.8382],
         [ 0.2989, -1.6392,  0.3325,  1.1027],
         [ 0.5152,  1.2363, -1.5415, -0.2332],
         [ 0.2993, -1.3430,  1.4365, -0.3147],
         [-1.0939,  1.3900, -0.8624,  0.5060],
         [ 0.4521, -1.4655, -0.2091,  1.2995]]], grad_fn=<AddBackward0>)


# And Viola!! We have the entire Encoder 🎉